In [29]:
import os
from datetime import date
from textblob import TextBlob
import pandas as pd
import mysql.connector as msql
import re
import nltk
import csv
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\shilt\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [30]:
search_term = "indihome"
from_date = "2021-09-10"
end_date = "2021-11-10"
max_results = 100

In [31]:
# Using OS library to call CLI commands in Python
tweets = 'snscrape --jsonl --max-results {} --since {} twitter-search "{} until:{}"> result-tweets.json'.format(max_results, from_date, search_term, end_date)
os.system(tweets)

# Reads the json generated from the CLI command above and creates a pandas dataframe
df = pd.read_json('result-tweets.json', lines=True)

In [32]:
def cleaner(tweets):
    tweets = " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweets).split())
    tweets = " ".join(re.sub(r"http\S+"," ", tweets).split())
    tweets = " ".join(w for w in nltk.wordpunct_tokenize(tweets))
    return tweets
df['content'] = df['content'].map(lambda x: cleaner(x))
df.to_csv('data.csv') #specify location
for row in df['content'].iteritems():
    print(row)

(0, 'AktivitasTanpaBatas InternetnyaIndonesia BerjuangTanpaBatas IndiHome')
(1, 'Jangan terlalu cepat menghakimi tanpa pernah mau mengerti sudut pandang yang lain Untuk urusan jaringan internet percayakan pada INDIHOME InternetnyaIndonesia AktivitasTanpaBatas')
(2, 'oh God i really HATE indihome')
(3, 'AktivitasTanpaBatas InternetnyaIndonesia BerjuangTanpaBatas IndiHome')
(4, 'tolong dong ini kenapa yah Apa ada maintenance')
(5, 'Siap Kak Rose Selamat menikmati layanan IndiHome dan menjalani aktivitas hari ini Semoga harinya menyenangkan Terima kasih Eshal')
(6, 'Dear bayar mahal kecepatannya wifinya yang tinggi tp tetep aja lemot wifinya')
(7, 'Halo Kak Ariya Terima kasih atas apresiasinya ya Apabila ada pertanyaan lain seputar layanan IndiHome silakan hubungi kami kembali Stay safe Stay healthy dan bersama IndiHome PastiBeres Syifa')
(8, 'Indihome astaga')
(9, 'Halo Kak Rose Terima kasih sudah berpartisipasi meramaikan kuis serta atas apresiasinya Semoga Kakak beruntung ya Stay safe 

In [33]:
df = pd.read_csv('data.csv', usecols=['content'])
df.to_csv('data_konten.csv') #specify location
df.head()

,content
0,AktivitasTanpaBatas InternetnyaIndonesia Berju...
1,Jangan terlalu cepat menghakimi tanpa pernah m...
2,oh God i really HATE indihome
3,AktivitasTanpaBatas InternetnyaIndonesia Berju...
4,tolong dong ini kenapa yah Apa ada maintenance


In [34]:
tweetData = pd.read_csv('data_konten.csv', index_col=False)
tweetData.head()

tweetData = msql.connect(host='localhost', user='root', passwd='', db='indihome_data')
cursor = tweetData.cursor()

csv_data = csv.reader(open('data_konten.csv'))
next(csv_data, None)
for row in csv_data:
    cursor.execute('INSERT INTO data(id, tweet) VALUES(%s, %s)', row)
tweetData.commit()
cursor.close()
print('Done')

Done


In [35]:
def translate(x):
    try:
        blob=TextBlob(x)
        return (str(blob.translate(to = 'en')))
    except: 
        return None

df=pd.read_csv('data_konten.csv')
df['content'] = df.content.astype(str) ##converting column to string where translation is required
df['content'] = df['content'].apply(lambda x: translate(x)) #Translation happening
df.to_csv('translated.csv') #specify location

In [36]:
def getSubjectifity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity
df['Subjectivity'] = df['content'].apply(getSubjectifity)
df['Polarity'] = df['content'].apply(getPolarity)
df

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'NoneType'>